In [1]:
import os

from keras import backend as K
# import numpy as np
from keras.models import load_model

with K.tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #saver = K.tf.train.Saver(write_version=K.tf.train.SaverDef.V1)
    
    out_file_name = 'new_transposed_unet_YCrCb_K03D02PaRnone_o__staged'
    keras_model_path = '/Users/pavelbulochkin/banuba/keras_models/alexander/new_transposed_unet_YCrCb_K03D02PaRnone_o__staged.hdf5'
    
    model = load_model(keras_model_path)
    model.load_weights(keras_model_path)
    
    in_tensor = model.input.name.split(':')[0]
    out_tensor = model.output.name.split(':')[0]

    out_folder_path = os.path.join(os.getcwd() + '/inference/')
    if not os.path.exists(out_folder_path) : 
        os.makedirs(out_folder_path)

    out_file_name_raw = out_file_name +'_raw' + '.pb'
    out_file_path_raw = os.path.join(out_folder_path, out_file_name_raw)
    
    out_file_name_freezed = out_file_name + '_freezed' + '.pb' 
    out_file_path_freezed = os.path.join(out_folder_path, out_file_name_freezed)

    out_file_name_for_inference =  out_file_name + '_for_inference' + '.pb'
    out_file_path_for_inference = os.path.join(out_folder_path, out_file_name_for_inference)

    ckpt_out_raw_file_path = os.path.join(out_folder_path, out_file_name_raw + '.ckpt')    
    
    saver = K.tf.train.Saver()   
    saver.save(sess, ckpt_out_raw_file_path)
    
    K.tf.train.write_graph(sess.graph.as_graph_def(), logdir=out_folder_path, name=out_file_name_raw)

    
tools_path = '/Users/pavelbulochkin/banuba/keras_models/tensorflow/bazel-bin/tensorflow/python/tools/'

#--input_binary=True
freeze_string = """{}/freeze_graph --input_graph={} --input_checkpoint={} --output_graph={} \
                    --output_node_names='{}' """.format(tools_path,
                                                    out_file_path_raw, 
                                                    ckpt_out_raw_file_path, 
                                                    out_file_path_freezed, 
                                                    out_tensor)

inferece_string = """{}/optimize_for_inference \
--input={} --output={}  --input_names='{}' \
--output_names='{}'""".format(tools_path, out_file_path_freezed, out_file_path_for_inference, in_tensor, out_tensor)

#print("In Tensor:", in_tensor)
#print("Out Tensor:", out_tensor)
print('-'*30)
print('Executing :', freeze_string)
print('-'*30)
print('Executing :', inferece_string)


Using TensorFlow backend.


------------------------------
('Executing :', "/Users/pavelbulochkin/banuba/keras_models/tensorflow/bazel-bin/tensorflow/python/tools//freeze_graph --input_graph=/Users/pavelbulochkin/banuba/keras_models/inference/new_transposed_unet_YCrCb_K03D02PaRnone_o__staged_raw.pb --input_checkpoint=/Users/pavelbulochkin/banuba/keras_models/inference/new_transposed_unet_YCrCb_K03D02PaRnone_o__staged_raw.pb.ckpt --output_graph=/Users/pavelbulochkin/banuba/keras_models/inference/new_transposed_unet_YCrCb_K03D02PaRnone_o__staged_freezed.pb                     --output_node_names='out_layer/div' ")
------------------------------
('Executing :', "/Users/pavelbulochkin/banuba/keras_models/tensorflow/bazel-bin/tensorflow/python/tools//optimize_for_inference --input=/Users/pavelbulochkin/banuba/keras_models/inference/new_transposed_unet_YCrCb_K03D02PaRnone_o__staged_freezed.pb --output=/Users/pavelbulochkin/banuba/keras_models/inference/new_transposed_unet_YCrCb_K03D02PaRnone_o__staged_for_inference.pb 

In [2]:
model.input

<tf.Tensor 'in_layer:0' shape=(?, 240, 320, 3) dtype=float32>

In [3]:
model.output

<tf.Tensor 'out_layer/div:0' shape=(?, 76800, 2) dtype=float32>